In [1]:
from GraphModule.gcn import MyGCN
from GraphModule.nettack import Nettack

In [2]:
#import pandas
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from deeprobust.graph.defense import GCN
#from deeprobust.graph.targeted_attack import Nettack
from deeprobust.graph.utils import accuracy
from deeprobust.graph.data import Dataset

No module named 'torch_geometric'
No module named 'torch_geometric'


/home/sols/graph_venv/lib/python3.8/site-packages/deeprobust/graph/defense/__init__.py:14: UserWarning: Please install pytorch geometric if you would like to use the datasets from pytorch geometric. See details in https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html
  warnings.warn("Please install pytorch geometric if you " +
/home/sols/graph_venv/lib/python3.8/site-packages/deeprobust/graph/data/__init__.py:9: UserWarning: Please install pytorch geometric if you would like to use the datasets from pytorch geometric. See details in https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html
  warnings.warn("Please install pytorch geometric if you " +


In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
def test_MyGCN(target_node, ori_adj, aug_adj, features, labels, train_idx, val_idx, test_idx):
    ''' test on GCN '''
    my_gcn = MyGCN(nfeat=features.shape[1], nhid=16, nclass=labels.max().item() + 1, dropout=0.5, device=device)

    my_gcn = my_gcn.to(device)

    my_gcn.fit(features, ori_adj, aug_adj, labels, train_idx, val_idx, patience=30)

    my_gcn.eval()
    output = my_gcn.predict()
    probs = torch.exp(output[[target_node]])[0]
    print('Target node probs: {}'.format(probs.detach().cpu().numpy()))
    print('Target node classification: {}'.format(probs.detach().cpu().numpy().argmax()))
    print('Target node labels: {}'.format(labels[target_node]))
    acc_test = accuracy(output[test_idx], labels[test_idx])

    print("Overall test set results:", "accuracy= {:.4f}".format(acc_test.item()))
    return acc_test.item()

In [4]:
data = Dataset(root='/tmp/', name='cora', seed=15)
adj, features, labels = data.adj, data.features, data.labels
idx_train, idx_value, idx_test = data.idx_train, data.idx_val, data.idx_test

idx_unlabeled = np.union1d(idx_value, idx_test)
target_node = 0
assert target_node in idx_unlabeled

Loading cora dataset...
Selecting 1 largest connected components


In [5]:
test_MyGCN(target_node, adj, adj, features, labels, idx_train, idx_value, idx_test)

Target node probs: [0.02927733 0.01206558 0.03448507 0.07881916 0.00389588 0.7676649
 0.07379214]
Target node classification: 5
Target node labels: 5
Overall test set results: accuracy= 0.8405


0.8405432595573441

In [6]:
surrogate = MyGCN(nfeat=features.shape[1], nhid=16, nclass=labels.max().item()+1, dropout=0.5, device=device)
surrogate.fit(features, adj, adj, labels, idx_train, idx_value)

In [7]:
model = Nettack(surrogate, nnodes=adj.shape[0], attack_structure=True, attack_features=True, device=device)
model = model.to(device)

In [10]:
degrees = adj.sum(0).A1
n_perturbations = int(degrees[target_node])

model.attack(features, adj, labels, target_node, n_perturbations, verbose=False)

In [11]:
modified_adj = model.modified_adj
modified_features = model.modified_features

In [13]:
test_MyGCN(target_node, adj, modified_adj, modified_features, labels, idx_train, idx_value, idx_test)

Target node probs: [0.04131257 0.00615348 0.0366347  0.0621196  0.03788184 0.6848579
 0.13103986]
Target node classification: 5
Target node labels: 5
Overall test set results: accuracy= 0.8325


0.8324949698189135